# Gui Mapper Basic Prototype (Perfect)

In [1]:
import pandas as pd
import json
from pandas.io.parsers import read_csv
import pickle

In [2]:
ground_truth_source = read_csv("GUI Mapping Ground Truth - Etsy.csv")
ground_truth_source

,id,xpath,class,canonical,current Activity,next Activity,type,additional info
0,com.etsy.android:id/btn_link,/hierarchy/android.widget.FrameLayout/android....,android.widget.Button,transition,.ui.homescreen.HomescreenTabsActivity,.ui.user.auth.SignInActivity,transition,"""Get Started"" button"
1,com.etsy.android:id/btn_sign_in_dialog,/hierarchy/android.widget.FrameLayout/android....,android.widget.Button,transition,.ui.user.auth.SignInActivity,.ui.user.auth.SignInActivity,transition,"""Sign In"" button"
2,com.etsy.android:id/edit_username,/hierarchy/android.widget.FrameLayout/android....,android.widget.EditText,signin_username_or_email,.ui.user.auth.SignInActivity,NaN,input,NaN
3,com.etsy.android:id/edit_password,/hierarchy/android.widget.FrameLayout/android....,android.widget.EditText,signin_password,.ui.user.auth.SignInActivity,NaN,input,NaN
4,com.etsy.android:id/button_signin,/hierarchy/android.widget.FrameLayout/android....,android.widget.Button,sign_in,.ui.user.auth.SignInActivity,NaN,assertion,NaN
5,com.etsy.android:id/forgot_password_link,/hierarchy/android.widget.FrameLayout/android....,android.widget.TextView,signin_forget_pwd,.ui.user.auth.SignInActivity,NaN,assertion,NaN


In [3]:
ground_truth_target = read_csv("GUI Mapping Ground Truth - Wish.csv")
ground_truth_target

,id,xpath,class,canonical,current Activity,next Activity,type,additional info
0,com.contextlogic.wish:id/sign_in_fragment_emai...,/hierarchy/android.widget.FrameLayout/android....,android.widget.EditText,signin_username_or_email,.activity.login.signin.SignInActivity,NaN,input,NaN
1,com.contextlogic.wish:id/sign_in_fragment_pass...,/hierarchy/android.widget.FrameLayout/android....,android.widget.EditText,signin_password,.activity.login.signin.SignInActivity,NaN,input,NaN
2,com.contextlogic.wish:id/sign_in_fragment_sign...,/hierarchy/android.widget.FrameLayout/android....,android.widget.TextView,sign_in,.activity.login.signin.SignInActivity,NaN,assertion,NaN
3,com.contextlogic.wish:id/sign_in_fragment_forg...,/hierarchy/android.widget.FrameLayout/android....,android.widget.TextView,signin_forget_pwd,.activity.login.signin.SignInActivity,NaN,assertion,NaN
4,NaN,/hierarchy/android.widget.FrameLayout/android....,android.widget.Button,transition,.activity.login.signin.SignInActivity,.activity.login.signin.SignInActivity,transition,ok button to transit to start


In [4]:
etsy_csv = read_csv("../src/test_csv/Etsy.csv", names=["method", "json"])
etsy_csv

,method,json
0,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."
1,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."
2,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."


In [5]:
def import_json(lst):
    lst = json.loads(lst)
    return [json.loads(str) for str in lst]

In [6]:
etsy_csv['json'] = etsy_csv['json'].apply(import_json)
etsy_csv

,method,json
0,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...
1,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...
2,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...


In [7]:
etsy_csv['json'][0]

[{'id_or_xpath': 'id@com.etsy.android:id/btn_link', 'action': 'click'},
 {'id_or_xpath': 'id@com.etsy.android:id/btn_link', 'action': 'click'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_username',
  'input': 'aaa',
  'action': 'sendKeys'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_password',
  'input': 'bbb',
  'action': 'sendKeys'},
 {'id_or_xpath': 'id@com.etsy.android:id/button_signin', 'action': 'click'}]

In [8]:
def perfect_map(test):
    for gui_event in test:
        if gui_event['id_or_xpath'][:3] == "id@":
            search_col = "id"
        else:
            search_col = "xpath"
        source_event = ground_truth_source.loc[ground_truth_source[search_col] == gui_event['id_or_xpath'][3:]]
        canonical = source_event.iloc[0]['canonical']
        gui_event['id_or_xpath'] = canonical
        if canonical == "transition":
            gui_event['id_or_xpath'] = "transition"
        else:
            target_event = ground_truth_target.loc[ground_truth_target['canonical'] == canonical]
            if pd.isnull(target_event.iloc[0]['id']):
                gui_event['id_or_xpath'] = target_event.iloc[0]['xpath']
            else:
                gui_event['id_or_xpath'] = target_event.iloc[0]['id']
    return test

In [9]:
wish_csv = pickle.loads(pickle.dumps(etsy_csv)) #deep copy
wish_csv['json'] = wish_csv['json'].apply(perfect_map)
wish_csv

,method,json
0,<Etsy.WelcomeTest: void testSignIn()>,"[{'id_or_xpath': 'transition', 'action': 'clic..."
1,<Etsy.WelcomeTest: void testSignIn()>,"[{'id_or_xpath': 'transition', 'action': 'clic..."
2,<Etsy.WelcomeTest: void testSignIn()>,"[{'id_or_xpath': 'transition', 'action': 'clic..."


In [10]:
wish_csv.iloc[0]['json']

[{'id_or_xpath': 'transition', 'action': 'click'},
 {'id_or_xpath': 'transition', 'action': 'click'},
 {'id_or_xpath': 'com.contextlogic.wish:id/sign_in_fragment_email_text',
  'input': 'aaa',
  'action': 'sendKeys'},
 {'id_or_xpath': 'com.contextlogic.wish:id/sign_in_fragment_password_text',
  'input': 'bbb',
  'action': 'sendKeys'},
 {'id_or_xpath': 'com.contextlogic.wish:id/sign_in_fragment_sign_in_button',
  'action': 'click'}]